In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import torch.nn.functional as F
import transformer_lens
from datasets import load_dataset, concatenate_datasets 
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/miguel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_and_merge_model(model_base: str, adapter_path: str):
    """
    Function to load a causal language model and merge an adapter.

    Args:
    model_base (str): Path or identifier of the base model (e.g. "EleutherAI/pythia-70m").
    adapter_path (str): Path to the fine-tuned adapter model.

    Returns:
    model_merged: The merged and unloaded model.
    tokenizer: The tokenizer loaded from the adapter.
    """

    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    model_id = AutoModelForCausalLM.from_pretrained(model_base)
    model = PeftModel.from_pretrained(model_id, adapter_path)    
    return model.merge_and_unload(), tokenizer

def process_activations(samples, model, num_layers, db_activations, task_name=None):
    """
    Process activations for a list of samples using a model and store the results in the provided activations database.

    Args:
    - samples (list): List of text samples to process.
    - model (object): Model with a `run_with_cache` method to obtain activations.
    - num_layers (int): Number of layers of the model to extract activations from.
    - db_activations (dict): Dictionary to store activations and task labels.
    - task_name (str): Name of the task to assign to the activations. Default is 'tel'.

    Returns:
    - db_activations (dict): Updated dictionary with activations and task labels.
    """
    
    for text in tqdm(samples, desc="Processing DB"):
        _, activations = model.run_with_cache(text)
        vector = []
        for layer in range(num_layers):
            block_act_fn = activations[f'blocks.{layer}.mlp.hook_post']
            vector.append(block_act_fn)
        vector = torch.cat(vector, dim=0).flatten().cpu().numpy()
        db_activations["activations"].append(vector)
        db_activations["task"].append(task_name)

    return db_activations 

In [3]:
model_base = "Qwen/Qwen2-1.5B-Instruct"

## Activations for math and tel

In [3]:
adapter_path = "fine_tuned_model_both_qwen"

#model_base = "EleutherAI/pythia-70m"
#adapter_path = "fine_tuned_model_both_pythia"

model_merged_tel, tokenizer_tel = load_and_merge_model(model_base, adapter_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
dataset_autoregressive = load_dataset('text', data_files={'train': 'data/sentences_train.txt'})
samples_tel = []
for sample in tqdm(dataset_autoregressive['train']['text']):
    if len(sample) > 0:
        # Remove all digits from the sample
        cleaned_sample = re.sub(r'\d+|\.', '', sample)
        samples_tel.append(cleaned_sample)

100%|██████████| 300/300 [00:00<00:00, 1041631.79it/s]


In [5]:
model_hooked = transformer_lens.HookedTransformer.from_pretrained(
    model_base,
    hf_model=model_merged_tel, 
    tokenizer=tokenizer_tel,
    device=device,
    move_to_device=True
)
num_of_layers = model_merged_tel.config.max_window_layers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model Qwen/Qwen2-1.5B-Instruct into HookedTransformer


In [6]:
db_activations = {
    "activations": [],
    "task": []
}

In [7]:
process_activations(samples_tel, model_hooked, num_of_layers, db_activations, task_name=0)

Processing DB:   0%|          | 0/150 [00:00<?, ?it/s]

Processing DB: 100%|██████████| 150/150 [00:11<00:00, 13.20it/s]


{'activations': [array([-0.00759343,  0.01179195,  0.04324816, ...,  0.00181106,
         -0.6167257 , -0.49666786], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.28923902,
         -0.8601406 ,  0.7978515 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.6773398 ,
         -0.02996855,  0.7834483 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.50649786,
         -0.14943008,  0.00720083], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.34183252,
         -0.99755704, -0.06896321], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.15167871,
         -0.22709699, -1.3994457 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.45771006,
         -0.09401266,  0.05321687], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.45504934,
         -0.15434039,  0.6567061 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.043

In [8]:
adapter_path = "fine_tuned_model_math_qwen"
model_merged_math, tokenizer_math = load_and_merge_model(model_base, adapter_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
dataset_math = load_dataset('json', data_files='./data/arithmatic_expressions.json', split="train")
print(dataset_math)
samples_math = []
for sample in tqdm(dataset_math['text']):
    samples_math.append(sample.split("=")[0] + "=")
samples_math = np.random.choice(samples_math, 150, replace=False)
print(len(samples_math))

Dataset({
    features: ['text'],
    num_rows: 2000
})


100%|██████████| 2000/2000 [00:00<00:00, 4488286.78it/s]

150


In [10]:
model_hooked_math = transformer_lens.HookedTransformer.from_pretrained(
    model_base,
    hf_model=model_merged_math, 
    tokenizer=tokenizer_math,
    device=device,
    move_to_device=True
)
num_of_layers = model_merged_tel.config.max_window_layers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model Qwen/Qwen2-1.5B-Instruct into HookedTransformer


In [11]:
process_activations(samples_math, model_hooked_math, num_of_layers, db_activations, task_name=1)

Processing DB: 100%|██████████| 150/150 [00:16<00:00,  9.30it/s]


{'activations': [array([-0.00759343,  0.01179195,  0.04324816, ...,  0.00181106,
         -0.6167257 , -0.49666786], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.28923902,
         -0.8601406 ,  0.7978515 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.6773398 ,
         -0.02996855,  0.7834483 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.50649786,
         -0.14943008,  0.00720083], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.34183252,
         -0.99755704, -0.06896321], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.15167871,
         -0.22709699, -1.3994457 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.45771006,
         -0.09401266,  0.05321687], dtype=float32),
  array([-0.00759343,  0.01179195,  0.04324816, ..., -0.45504934,
         -0.15434039,  0.6567061 ], dtype=float32),
  array([-0.00759343,  0.01179195,  0.043

In [12]:
FOLDER = "results"
os.makedirs(FOLDER, exist_ok=True)  # Crear la carpeta si no existe

# Guardar el diccionario db_activations en formato pickle
with open(os.path.join(FOLDER, "activations.pkl"), 'wb') as file:
    pickle.dump(db_activations, file)

In [13]:
#delete model_hooked
del model_hooked
# clean up memory
torch.cuda.empty_cache()

## Activations for maths

In [4]:
adapter_path = "fine_tuned_model_math_qwen"
model_merged_math, tokenizer_math = load_and_merge_model(model_base, adapter_path)
db_activations = {
    "activations": [],
    "task": [],
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
dataset_math = load_dataset('json', data_files='./data/arithmatic_expressions.json', split="train")
print(dataset_math)
samples_sum = []
samples_sub = []
samples_mult = []
samples_div = []
for sample in tqdm(dataset_math['text']):
    if "+" in sample:
        samples_sum.append(sample.split("=")[0] + "=")
    elif "-" in sample:
        samples_sub.append(sample.split("=")[0] + "=")
    elif "x" in sample:
        samples_mult.append(sample.split("=")[0] + "=")
    elif "/" in sample:
        samples_div.append(sample.split("=")[0] + "=")
    else:
        raise ValueError(f"Sample {sample} does not contain any operator.")
    
MAX_SAMPLES = 100
samples_sum = np.random.choice(samples_sum, MAX_SAMPLES, replace=False)
samples_sub = np.random.choice(samples_sub, MAX_SAMPLES, replace=False)
samples_mult = np.random.choice(samples_mult, MAX_SAMPLES, replace=False)
samples_div = np.random.choice(samples_div, MAX_SAMPLES, replace=False)

Dataset({
    features: ['text'],
    num_rows: 2000
})


100%|██████████| 2000/2000 [00:00<00:00, 3739905.48it/s]


In [6]:
import transformer_lens.utils as utils
device = utils.get_device()

model_hooked_math = transformer_lens.HookedTransformer.from_pretrained(
    model_base,
    hf_model=model_merged_math, 
    tokenizer=tokenizer_math,
    device=device,
)
num_of_layers = model_merged_math.config.max_window_layers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model Qwen/Qwen2-1.5B-Instruct into HookedTransformer


In [7]:
process_activations(samples_sum, model_hooked_math, num_of_layers, db_activations, task_name=0)
process_activations(samples_sub, model_hooked_math, num_of_layers, db_activations, task_name=1)
process_activations(samples_mult, model_hooked_math, num_of_layers, db_activations, task_name=2)
process_activations(samples_div, model_hooked_math, num_of_layers, db_activations, task_name=3)

Processing DB: 100%|██████████| 100/100 [00:06<00:00, 14.88it/s]


{'activations': [array([-0.01007269,  0.00645165,  0.03944412, ..., -0.06560262,
         -0.43739694, -1.1813518 ], dtype=float32),
  array([-1.0072688e-02,  6.4516482e-03,  3.9444115e-02, ...,
         -2.0597950e-03, -5.0576675e-01, -4.6582303e+00], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ..., -0.29624072,
         -0.1931593 , -4.610728  ], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ..., -0.2555054 ,
         -0.11925691, -3.3747988 ], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ..., -0.1797839 ,
         -0.18773931, -1.473257  ], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ...,  0.07989857,
         -0.34788406, -3.4923775 ], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ..., -0.02832895,
         -0.3493095 , -1.8100919 ], dtype=float32),
  array([-0.01007269,  0.00645165,  0.03944412, ..., -0.07027873,
         -0.22028182, -2.2835083 ], dtype=float32),
  array([-0.01007269,  

In [8]:
# delete model_hooked
del model_hooked_math
del model_merged_math
# clean up memory
torch.cuda.empty_cache()

In [9]:
FOLDER = "results"
os.makedirs(FOLDER, exist_ok=True)  # Crear la carpeta si no existe

# Guardar el diccionario db_activations en formato pickle
with open(os.path.join(FOLDER, "activations_math.pkl"), 'wb') as file:
    pickle.dump(db_activations, file)

## Activation for tec and sports

In [4]:
adapter_path = "models/fine_tuned_model_st_qwen"
model_merged_math, tokenizer_math = load_and_merge_model(model_base, adapter_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
#peparacion dataset tecnologia y deporte
SAMPLES = 50
ds_sports = load_dataset('json', data_files='./data/sports_phrases.json', split="train")
# saplme of 100
ds_sports = ds_sports.shuffle(seed=42).select(range(SAMPLES))
print(ds_sports)
ds_technology = load_dataset('json', data_files='./data/technology_phrases.json', split="train")
# saplme of 100
ds_technology = ds_technology.shuffle(seed=42).select(range(SAMPLES))
print(ds_technology)
ds_technology_and_sports = load_dataset('json', data_files='./data/sports_and_technology_phrases.json', split="train")
print(ds_technology_and_sports)

Dataset({
    features: ['text'],
    num_rows: 50
})
Dataset({
    features: ['text'],
    num_rows: 50
})
Dataset({
    features: ['text'],
    num_rows: 50
})


In [6]:
import transformer_lens.utils as utils
device = utils.get_device()

model_hooked_math = transformer_lens.HookedTransformer.from_pretrained(
    model_base,
    hf_model=model_merged_math, 
    tokenizer=tokenizer_math,
    device=device,
)
num_of_layers = model_merged_math.config.max_window_layers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model Qwen/Qwen2-1.5B-Instruct into HookedTransformer


In [7]:
db_activations = {
    "activations": [],
    "task": [],
}

process_activations(ds_sports['text'], model_hooked_math, num_of_layers, db_activations, task_name=0)
process_activations(ds_technology['text'], model_hooked_math, num_of_layers, db_activations, task_name=1)
#process_activations(ds_technology_and_sports['text'], model_hooked_math, num_of_layers, db_activations, task_name=2)

Processing DB: 100%|██████████| 50/50 [00:03<00:00, 13.79it/s]


{'activations': [array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.04947617,
         -0.4928999 ,  0.19141941], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.17227168,
         -0.28218508, -0.33614278], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.17355634,
         -0.27070752, -0.33200464], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.1477541 ,
         -0.34323332, -0.51155573], dtype=float32),
  array([-0.0061341 ,  0.0089793 ,  0.05091259, ...,  0.11383496,
         -0.3483282 , -0.22542438], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.02516714,
         -0.37821907, -0.05760647], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.08691163,
         -0.31890917, -0.23929344], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.06987438,
         -0.34479555, -0.0980793 ], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.050

In [8]:
FOLDER = "results"
os.makedirs(FOLDER, exist_ok=True)  # Crear la carpeta si no existe

# Guardar el diccionario db_activations en formato pickle
with open(os.path.join(FOLDER, "activations_st.pkl"), 'wb') as file:
    pickle.dump(db_activations, file)

In [9]:
db_activations = {
    "activations": [],
    "task": [],
}

process_activations(ds_technology_and_sports['text'], model_hooked_math, num_of_layers, db_activations, task_name=None)

Processing DB: 100%|██████████| 50/50 [00:03<00:00, 15.02it/s]


{'activations': [array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.05420539,
         -0.37763283, -0.17196988], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.20489185,
         -0.23333201, -0.16990888], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.11328325,
         -0.34368584,  0.06578713], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.00928913,
         -0.3864812 ,  0.22273508], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.11818854,
         -0.3668709 ,  0.047583  ], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.06440731,
         -0.40862224, -0.06646411], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.15581055,
         -0.2766421 , -0.15724462], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.05091265, ...,  0.08972092,
         -0.30560353,  0.20197658], dtype=float32),
  array([-0.00613406,  0.0089793 ,  0.050

In [10]:
FOLDER = "results"
os.makedirs(FOLDER, exist_ok=True)  # Crear la carpeta si no existe

# Guardar el diccionario db_activations en formato pickle
with open(os.path.join(FOLDER, "activations_s_and_t.pkl"), 'wb') as file:
    pickle.dump(db_activations, file)